In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model, model_from_json
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.models import Model
from keras import backend as K
import keras
from sklearn.metrics import log_loss
from imutils import paths
import os
import seaborn
import cv2
import random
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

from load_cifar10 import load_cifar10_data

#from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [2]:
# create the base pre-trained model
def inception_v3_model(nb_classes):
    base_model = InceptionV3(weights='imagenet', include_top=False)

    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    # and a logistic layer
    predictions = Dense(nb_classes, activation='softmax')(x)

    # this is the model we will train
    model = Model(input=base_model.input, output=predictions)

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
    for layer in base_model.layers:
        layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
    print ("starting model compile")
    compile(model)
    print ("model compile done")

    return model 

In [3]:
def compile(model):
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["accuracy"])

In [12]:
img_rows, img_cols = 299, 299 # Resolution of inputs
channel = 3
num_classes = 2 
batch_size = 16 
nb_epoch = 10

In [13]:
#X_train, Y_train, X_valid, Y_valid = load_cifar10_data(img_rows, img_cols)

In [14]:
#X_train

In [15]:
data = []
labels = []

imagePaths = sorted(list(paths.list_images('/home/jailsonpereira/PAIC_CNN/dataset')))
random.seed(42)
random.shuffle(imagePaths)

for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image,(299,299))
    data.append(image)
    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    

data = np.array(data,dtype='float32')/255
labels = np.array(labels)
print(data.shape)

(223, 299, 299, 3)


In [16]:
le=LabelEncoder()
labels=le.fit_transform(labels)
labels=to_categorical(labels,num_classes=2)

In [17]:
X_train,X_val,Y_train,Y_val=train_test_split(data,labels,test_size=0.1)

In [18]:
X_train = X_train.reshape(X_train.shape[0], channel, img_rows, img_cols)

In [19]:
X_val = X_val.reshape(X_val.shape[0], channel, img_rows, img_cols)

In [21]:
X_val.shape

(23, 3, 299, 299)

In [20]:
model = inception_v3_model(num_classes)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["accuracy"])

starting model compile
model compile done


/home/jailsonpereira/anaconda3/envs/paic2/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [22]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, None, None 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, None, Non 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, None, Non 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, None, Non 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [24]:
model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              shuffle=True,
              verbose=1,
              validation_data=(X_val, Y_val),
         )

/home/jailsonpereira/anaconda3/envs/paic2/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 200 samples, validate on 23 samples
Epoch 1/10
200/200 [==============================] - 153s 766ms/step - loss: 4.2840 - acc: 0.6800 - val_loss: 4.9055 - val_acc: 0.6957
Epoch 2/10
200/200 [==============================] - 157s 783ms/step - loss: 4.3519 - acc: 0.7300 - val_loss: 4.9055 - val_acc: 0.6957
Epoch 3/10
200/200 [==============================] - 160s 801ms/step - loss: 4.3519 - acc: 0.7300 - val_loss: 4.9055 - val_acc: 0.6957
Epoch 4/10
200/200 [==============================] - 158s 788ms/step - loss: 4.3519 - acc: 0.7300 - val_loss: 4.9055 - val_acc: 0.6957
Epoch 5/10
200/200 [==============================] - 157s 785ms/step - loss: 4.3519 - acc: 0.7300 - val_loss: 4.9055 - val_acc: 0.6957
Epoch 6/10
200/200 [==============================] - 163s 817ms/step - loss: 4.3519 - acc: 0.7300 - val_loss: 4.9055 - val_acc: 0.6957
Epoch 7/10
200/200 [==============================] - 158s 792ms/step - loss: 4.3519 - acc: 0.7300 - val_loss: 4.9055 - val_acc: 0.6957
Epo